![DLI Header](images/DLI_Header.png)

# Chatbot

In this notebook you will begin to create chatbot functionality, creating an AI bot capable of retaining conversation history.

## Learning Objectives

By the time you complete this notebook you will be able to:
- Create chatbot functionality from our LLaMA-2 model, capable of retaining conversation history.

## Video Walkthrough

Execute the cell below to load the video walkthrough of this notebook.

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/06-chatbot.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## Create LLaMA-2 Pipeline

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## Helper Functions and Classes

In this notebook we will use the following functions and classes to support our interaction with the LLM. Feel free to skim over them presently, as they are covered in greater detail when used below.

### Generate Model Responses

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [ ]:
class LlamaChatbot:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
    - system_context (str): Contextual information to provide to the language model for all conversations.
    - conversation_history (list of tuples): Stores the history of the conversation, where each
      tuple contains a user message and the corresponding agent response.
    """

    def __init__(self, system_context):
        """
        Initializes a new instance of the LlamaChatbot class.

        Parameters:
        - system_context (str): A string that sets the initial context for the language model.
        """
        self.system_context = system_context
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history.

        Parameters:
        - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
        - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

## No Conversation Memory

Let's strike up a conversation with our LLM. We are going to provide it a **system context** that it should be a friendly chatbot, and (naively) encourage it to always remember our name if provided in a conversation.

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Hello my name is Star. Nice to meet you!"

print(generate(construct_prompt_with_context(prompt, system_context)))

---

The model certainly appears to be eager to show that it remembers who we are. Let's see what happens when we actually put its name-retention to the test.

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Can you remind me what my name is?"

print(generate(construct_prompt_with_context(prompt, system_context)))

---

It likely comes as no surprise that the model does not remember our name because in spite of how it presents itself to us, we have provided it with no capacity to remember any of the details from previous conversational exchanges. The model seems to insist that our name is "Emily," which is clearly incorrect. When models generate responses that are fabricated, often with confidence, we call this **hallucination**.

## Create Conversation Memory

In order to create a chatbot experience where the model can retain information from previous exchanges, we are going to use a `LlamaChatbot` class (defined above). Here's the `help` output from our class definition.

In [ ]:
help(LlamaChatbot)

---

Most pertinent to our immediate goal is the creation of a `conversation_history` list that we will append to anytime we invoke the `chat` method. We will be reusing some of the same logic from previous notebooks, in particular leveraging the LLaMA-2 **prompt template**, so that each user/model interaction if formatted properly, and then prepended to the prompt for subsequent exchanges between the user and the model.

It would be accurate to say that with each interaction, we are performing **few-shot learning** where the instructive examples are simply the previous interactions.

Let's see it in action.

In [ ]:
system_context = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. You are always kind \
but also repectful and professional.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat("Hi, my name is Star. Nice to meet you!"))

---

So far so good. Let's see now if the model is able to "recall" our name.

In [ ]:
print(chatbot.chat("Can you remind me what my name is?"))

---

Success! Let's take a look at the model's conversation history.

In [ ]:
chatbot.conversation_history

---

Given that `conversation_history` is prepended to each new prompt, it makes sense that the model is able to generate responses based on previous exchanges.

The `reset` method will clear `conversation_history`.

In [ ]:
chatbot.reset()

In [ ]:
print(chatbot.chat("Can you remind me what my name is?"))

---

And now it is no surprise that the model is unable to "recall" details from our previous exchanges.

## Exercise: Task Tracker

Create an assistant that can keep track of what you need to get done today. It should be able to add and remove things from your list based on your dialogue, and at any given time, accurately remind you of what you have left to do.

See the solution below if you get stuck.

### Your Work Here

## Solution

In [ ]:
system_context = """
You are an assistant that help me keep track of what I need to do. You keep track of tasks that I \
provide you, and when asked remind me of what I have left to do.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat("I need to do the following things today: eat breakfast, eat lunch, eat dinner, go to work, exercise, and clean the house"))

In [ ]:
print(chatbot.chat("Yes, I also need to hang out with friends."))

In [ ]:
print(chatbot.chat("Okay, I'm done eating breakfast and exercising."))

In [ ]:
print(chatbot.chat("I've eaten lunch. Sometime today I need to call the bike shop."))

In [ ]:
print(chatbot.chat("I finished work, hung out with friends, cleaned the house, and called the bike shop."))

In [ ]:
print(chatbot.chat("I just ate dinner. Now I just need to go to sleep."))

In [ ]:
chatbot.reset()

## Key Concept Review

The following key concepts were introduced in this notebook:

- **Hallucination:** When a model generates, often with some expressed confidence, untrue or inaccurate responses.

## Optional Advanced Exercises

**NOTE:** In the next notebook we are going to discover limits to the amount of conversation the model is able to store before things start to go wrong. With that in mind, and before suggesting additional experimentation, if you find that the model is starting to produce only empty responses, move on to the next section so you can learn what this is all about.

If you'd like to go above and beyond the requirements of the course, below are some additional open-ended exercises for you to try.

### Use the 7B Model

At the top of the notebook, after restarting the kernel (see cell below), uncomment and use the 7B model instead of the 13B model we demoed. Try to get satisfying results in spite of using the smaller (weaker) model.

### Make a Helper Bot

Make a bot that someone can use when they are having a hard day, that gives encouragement, praise, and empathy, and knows when to respond and when to ask for more questions about what is going on for the user.

## Restart the Kernel

In order to free up GPU memory for the next notebook, please run the following cell to restart the kernel.

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

![DLI Header](images/DLI_Header.png)